## Preambule

In [138]:
import numpy as np
import pandas as pd
import yaml
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
from importlib import reload

## Run class

In [139]:
import Classes
reload(Classes)
from Classes import ecemf_reader, calculator

In [140]:
readclass = ecemf_reader()
readclass.read_csv_files()
readclass.add_average_gdps()
readclass.convert_to_xr()

x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:53: RuntimeWarning: Mean of empty slice.
  ag = (np.array(self.pd_mod[(self.pd_mod.Variable == 'GDP|PPP') & (self.pd_mod.Model == m) & (self.pd_mod.Region == 'Europe')])[:, 5:]).mean(axis=0)
x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:61: RuntimeWarning: Mean of empty slice.
  ag = (np.array(self.pd_mod[(self.pd_mod.Variable == 'GDP|PPP') & (self.pd_mod.Model == m) & (self.pd_mod.Region == 'World')])[:, 5:]).mean(axis=0)
x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:70: RuntimeWarning: Mean of empty slice
  av_gdp[t_i, 0] = np.nanmean(np.array(all_gdpse)[:, t_i].astype(float))
x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:72: RuntimeWarning: Mean of empty slice
  av_gdp[t_i, 1] = np.nanmean(np.array(all_gdpsw)[:, t_i].astype(float))


In [141]:
calcclass = calculator(readclass.data_xr)
calcclass.calculate_responsiveness_indicators()
calcclass.calculate_mitigationstrategy_indicators()
calcclass.calculate_energysupply_indicators()
calcclass.calculate_energydemand_indicators()
calcclass.calculate_costandeffort_indicators()
calcclass.convert_to_indicator_xr()
calcclass.export_indicators_to_netcdf()

x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:171: RuntimeWarning: All-NaN slice encountered
  potentials[s_i, m_i] = np.nanmax(speed[s_i, m_i], axis=1)
x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:194: RuntimeWarning: Mean of empty slice
  sens_prim = np.nanmean(var_prim, axis=1)
x:\user\dekkerm\Projects\IAMfingerprints\Calculating\Classes.py:216: RuntimeWarning: Mean of empty slice
  sens_dem = np.nanmean(var_dem, axis=1)
